In [ ]:
import numpy as np
import concurrent.futures
import time

def partition_and_retrieve_subsets(weights):
    total_sum = sum(weights)
    if total_sum % 2 != 0:  # Проверяем сумму элементов массива на четность
        return False, None, None

    n = len(weights)
    target = total_sum // 2  # Вычисляем целевую сумму, равную половине суммы элементов исходного массива
    dp = np.zeros((n + 1, target + 1), dtype=bool)  # Создаем двумерный массив dp размером (n+1) x (target+1) и заполняем нулями

    dp[0][0] = True

    for i in range(1, n + 1):  # Проходим по индексам массива weights от 1 до n
        weight = weights[i - 1]  # Получаем вес текущего элемента
        for j in range(target + 1):  # Проходим по возможным значениям суммы от 0 до target
            dp[i][j] = dp[i - 1][j]  # Записываем значение из предыдущей строки
            if j >= weight:  # Проверяем, можно ли добавить текущий элемент в подмножество суммы j
                dp[i][j] = dp[i][j] or dp[i - 1][j - weight]  # Устанавливаем значение dp[i][j] в зависимости от веса текущего элемента

    if not dp[n][target]:  # Проверяем, достижима ли целевая сумма
        return False, None, None

    subset1, subset2 = [], []
    i, j = n, target
    while i > 0:
        if dp[i][j] and not dp[i - 1][j]:
            subset1.append(weights[i - 1])  # Добавляем элемент в первое подмножество
            j -= weights[i - 1]  # Уменьшаем целевую сумму
        else:
            subset2.append(weights[i - 1])  # Добавляем элемент во второе подмножество
        i -= 1

    return True, sorted(subset1), sorted(subset2)  # Возвращаем True и два подмножества

def solve_partition_problems(weights_list):
    results_dict = {}
    with concurrent.futures.ProcessPoolExecutor() as executor:  # Создаем экземпляр ProcessPoolExecutor
        futures = {executor.submit(partition_and_retrieve_subsets, weights): weights for weights in weights_list}  # Создаем словарь с Future для каждого массива в weights_list
        for future in concurrent.futures.as_completed(futures):
            weights = futures[future]
            results_dict[tuple(weights)] = future.result()
    return results_dict

# Тестирование на наборе задач
weights_list = [
    [1, 5, 11, 5],
    [1, 2, 3, 4],
    [3, 3, 3, 3],
    [1, 2, 3, 1],
    [4, 4, 4, 4, 4],
    [1, 100, 50, 20, 29],
    [46, 61, 82, 8, 41, 14, 2, 82, 17, 16, 20, 27, 26, 81, 51, 38, 60, 64, 18, 1, 55, 50, 72, 93, 38, 2, 48, 31, 95, 91, 27, 6, 27, 97, 49, 96, 90, 81, 38, 21, 34, 89, 24, 28, 77, 64, 89, 10, 28, 87, 32, 88, 52, 81, 5, 31, 68, 83, 95, 27, 58, 18, 22, 35, 84, 90, 94, 77, 80, 7, 1, 7, 81, 53, 90, 79, 8, 92, 80, 67, 97, 51, 73, 43, 67, 102, 33, 49, 17, 94, 75, 34, 5, 40, 34, 16, 77, 89, 57, 99, 71, 21, 88, 17, 94, 34, 64, 97, 38, 56, 71, 30, 7, 44, 63, 10, 73, 4, 96, 66, 76, 28, 8, 2, 79, 49, 33, 24, 45, 59, 47, 12, 34, 48, 16, 43, 78, 58, 79, 98, 50, 52, 93, 99, 29, 94, 97, 50, 64, 18, 80, 25, 79, 27, 90, 41, 16, 18, 59, 18, 89, 1, 82, 37, 31, 22, 84, 84, 52, 12, 26, 95, 40, 69, 97, 91, 82, 91, 3, 98, 20, 62, 20, 59, 36, 71, 13, 53, 19, 88, 16, 54, 34, 59, 53, 51, 77, 69, 56, 20, 100, 75, 4, 68, 20, 93, 54, 96, 98, 97, 29, 28, 39, 78, 90, 30, 46, 59, 8, 5, 57, 54, 85, 4, 26, 24, 6, 96, 44, 49, 1, 22, 7, 51, 24, 100, 67, 25, 51, 30, 87, 94, 13, 97, 79, 23, 98, 33, 45, 1, 105, 95, 24, 93, 58, 17, 33, 35, 73, 47, 23, 93, 45, 49, 5, 25, 97, 43, 75, 11, 35, 24, 41, 33, 11, 81, 44, 22, 37, 65, 36, 83, 20, 4, 91, 55, 40, 100, 63, 33, 87, 19, 89, 68, 93, 65, 65, 23, 69, 56],
    [41, 64, 36, 45, 98, 52, 32, 1, 11, 70, 66, 2, 30, 21, 83, 50, 51, 73, 61, 14, 20, 88, 79, 122, 54, 47, 7, 51, 50, 166, 75, 29, 121, 35, 62, 89, 54, 74, 16, 62, 56, 80, 44, 4, 49, 96, 58, 53, 58, 36, 15, 96, 82, 84, 42, 89, 58, 77, 34, 44, 43, 12, 167, 38, 8, 48, 5, 68, 100, 91, 66, 14, 62, 23, 55, 64, 66, 69, 96, 16, 67, 21, 17, 91, 52, 39, 57, 55, 97, 84, 94, 36, 47, 26, 73, 70, 31, 12, 53, 62],
    [56, 21, 87, 18, 11, 57, 24, 47, 98, 99, 67, 60, 55, 65, 43, 9, 100, 43, 1, 57, 9, 98, 63, 15, 18, 86, 63, 17, 87, 99, 77, 9, 39, 26, 73, 25, 90, 65, 74, 5, 80, 36, 27, 40, 12, 29, 73, 76, 43, 27, 7, 90, 14, 6, 92, 10, 26, 56, 93, 95, 1, 6, 70, 4, 35, 75, 73, 19, 69, 100, 91, 98, 42, 99, 55, 63, 61, 80, 66, 67, 85, 9, 34, 19, 45, 44, 14, 89, 74, 91, 76, 58, 18, 41, 69, 6, 22, 50, 32, 14, 21, 16, 75, 78, 43, 1, 60, 76, 69, 23, 60, 53, 8, 87, 7, 76, 41, 100, 4, 1, 98, 71, 33, 87, 96, 88, 79, 40, 87, 54, 69, 11, 13, 39, 96, 68, 57, 31, 7, 19, 72, 1, 89, 71, 100, 46, 67, 67, 52, 92, 67, 38, 51, 4, 25, 74, 65, 63, 80, 83, 33, 53, 30, 72, 49, 67, 26, 38, 67, 77, 65, 88, 90, 85, 25, 68, 97, 95, 76, 28, 23, 69, 29, 10, 86, 29, 59, 12, 78, 78, 68, 85, 94, 61, 58, 59, 19, 35, 52, 88, 86, 38, 18, 35, 75, 20, 13, 48, 62, 57, 51, 26, 87, 27, 25, 24, 48, 35, 8, 79, 100, 27, 23, 42, 30, 26, 81, 44, 15, 35, 50, 54, 67, 5, 12, 51, 37, 33, 17, 44, 29, 86, 10, 63, 22, 79, 27, 121, 81, 14, 65, 78, 57, 96, 33, 12, 100, 6, 36, 71, 54, 60, 73, 69, 89, 9, 96, 32, 78, 79, 23, 65, 37, 47, 18, 34, 85, 92, 17, 9, 75, 5, 98, 58, 41, 37, 6, 21, 29, 80, 60, 80, 47, 88, 26, 38, 59, 66, 4, 63],
    [28, 27, 86, 97, 10, 59, 7, 115, 95, 24, 91, 35, 12, 77, 80, 88, 38, 96, 73, 18, 70, 87, 87, 1, 20, 76, 96, 34, 82, 95, 15, 67, 67, 38, 24, 5, 86, 54, 35, 48, 89, 22, 98, 64, 81, 122, 8, 6, 60, 77, 69, 75, 65, 34, 21, 21, 30, 41, 19, 41, 6, 39, 95, 79, 18, 68, 39, 60, 78, 76, 69, 98, 50, 34, 95, 72, 33, 70, 21, 80, 79, 65, 32, 41, 70, 42, 82, 15, 70, 32, 39, 5, 100, 86, 78, 16, 65, 46, 88, 89, 92, 34, 46, 25, 35, 55, 77, 85, 51, 85, 40, 52, 70, 4, 24, 24, 8, 39, 34, 34, 75, 4, 36, 85, 95, 20, 38, 69, 61, 50, 40, 14, 12, 3, 111, 19, 4, 69, 77, 43, 28, 43, 78, 74, 8, 37, 55, 41, 68, 72, 4, 12, 16, 30, 99, 97, 99, 65, 65, 42, 66, 72, 94, 46, 91, 78, 43, 41, 90, 33, 34, 20, 76, 49, 24, 35, 40, 75, 64, 84, 15, 27, 45, 7, 53, 76, 85, 18, 89, 96, 70, 56, 18, 100, 95, 91, 44, 86, 84, 86],
    [4, 55, 60, 7, 62, 86, 40, 36, 97, 29, 50, 40, 20, 93, 59, 54, 65, 40, 70, 3] ,
    [25, 60, 26, 81, 60, 75, 68, 11, 73, 18, 8, 38, 98, 86, 96, 21, 55, 21, 53, 5] ,
    [122, 99, 59, 47, 92, 24, 85, 99, 5, 113, 6, 85, 79, 94, 55, 68, 3, 162, 76, 39] ,
    [95, 108, 64, 81, 58, 73, 28, 20, 39, 28, 5, 70, 78, 51, 100, 73, 32, 70, 91, 74] ,
    [75, 87, 16, 36, 45, 86, 73, 93, 150, 28, 122, 41, 29, 5, 91, 47, 41, 73, 4, 34] ,
    [28, 79, 81, 75, 89, 67, 42, 53, 47, 38, 76, 91, 58, 74, 78, 53, 10, 14, 18, 65] ,
    [30, 31, 40, 19, 90, 88, 49, 3, 96, 92, 81, 70, 27, 86, 1, 90, 68, 96, 61, 78] ,
    [25, 158, 30, 12, 31, 91, 19, 26, 9, 84, 76, 69, 45, 16, 65, 18, 76, 46, 80, 188] ,
    [56, 25, 97, 65, 36, 40, 46, 82, 31, 116, 7, 47, 66, 40, 8, 38, 74, 97, 87, 74] ,
    [67, 80, 34, 98, 46, 3, 43, 141, 120, 58, 92, 45, 20, 96, 90, 108, 72, 63, 89, 3] ,
    [90, 69, 63, 23, 2, 67, 97, 66, 33, 27, 59, 48, 54, 33, 34, 41, 98, 34, 58, 104] ,
    [73, 71, 93, 82, 52, 76, 55, 54, 100, 24, 90, 37, 70, 214, 28, 75, 40, 65, 66, 97] ,
    [18, 61, 51, 91, 98, 76, 58, 20, 74, 34, 66, 15, 68, 3, 26, 91, 41, 44, 41, 100] ,
    [12, 30, 46, 10, 40, 20, 89, 69, 53, 99, 29, 18, 41, 61, 55, 68, 72, 86, 82, 78] ,
    [42, 72, 28, 30, 41, 97, 16, 53, 17, 50, 9, 39, 47, 42, 17, 96, 71, 35, 50, 60] ,
    [73, 6, 55, 64, 15, 222, 28, 12, 97, 78, 103, 13, 83, 17, 37, 68, 75, 44, 28, 90] ,
    [13, 93, 149, 2, 72, 6, 3, 25, 16, 96, 43, 80, 9, 40, 9, 36, 95, 97, 38, 12] ,
    [71, 67, 93, 19, 138, 78, 43, 92, 98, 42, 19, 111, 13, 95, 1, 89, 63, 55, 9, 48] ,
    [90, 11, 56, 72, 2, 133, 35, 59, 36, 23, 37, 95, 82, 68, 2, 40, 19, 32, 90, 30] ,
    [63, 95, 78, 46, 98, 50, 33, 47, 18, 17, 16, 96, 61, 49, 49, 94, 65, 64, 45, 64]

    ]

if __name__ == '__main__':
    start_time = time.time()
    results = solve_partition_problems(weights_list)
    end_time = time.time()
    for weights, result in results.items():
        can_part, subset1, subset2 = result
        if can_part:
            print(f"Массив {weights} можно разделить на два подмножества с одинаковой суммой. Сумма массива:", sum(weights))
            print("Подмножество 1:", subset1, "Сумма 1:", sum(subset1))
            print("Подмножество 2:", subset2, "Сумма 2:", sum(subset2),"\n")
        else:
            print(f"Массив {weights} невозможно разделить на два подмножества с одинаковой суммой.","\n")
    print(f"Время выполнения: {end_time - start_time:.4f} секунд")


Массив (1, 5, 11, 5) можно разделить на два подмножества с одинаковой суммой. Сумма массива: 22
Подмножество 1: [11] Сумма 1: 11
Подмножество 2: [1, 5, 5] Сумма 2: 11 

Массив (1, 2, 3, 4) можно разделить на два подмножества с одинаковой суммой. Сумма массива: 10
Подмножество 1: [2, 3] Сумма 1: 5
Подмножество 2: [1, 4] Сумма 2: 5 

Массив (1, 2, 3, 1) невозможно разделить на два подмножества с одинаковой суммой. 

Массив (3, 3, 3, 3) можно разделить на два подмножества с одинаковой суммой. Сумма массива: 12
Подмножество 1: [3, 3] Сумма 1: 6
Подмножество 2: [3, 3] Сумма 2: 6 

Массив (4, 4, 4, 4, 4) невозможно разделить на два подмножества с одинаковой суммой. 

Массив (1, 100, 50, 20, 29) можно разделить на два подмножества с одинаковой суммой. Сумма массива: 200
Подмножество 1: [100] Сумма 1: 100
Подмножество 2: [1, 20, 29, 50] Сумма 2: 100 

Массив (41, 64, 36, 45, 98, 52, 32, 1, 11, 70, 66, 2, 30, 21, 83, 50, 51, 73, 61, 14, 20, 88, 79, 122, 54, 47, 7, 51, 50, 166, 75, 29, 121, 35, 